# **The Battle of the Neighborhoods - Week 2**

Part 2 Web scrapping of Neigborhood, Zip Codes and Covid 19 information

**A : NEIGBORHOOD AND ZIP CODES DATA**

Web scrapping of Population data from Health New York - https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm

Download all the dependencies that is needed.

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: | 

**Web scrapping of Population data from wikipedia page using BeautifulSoup.**

Beautiful Soup is a Python package for parsing HTML and XML documents (including having malformed markup, i.e. non-closed tags, so named after tag soup). It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [ ]:
source = requests.get('https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm').text
soup=BeautifulSoup(source,'html')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

In [ ]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

In [ ]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [ ]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

**Tranform the data into a pandas dataframe**

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. Start by creating an empty dataframe.

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then loop through the data and fill the dataframe one row at a time

In [ ]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

In [ ]:
df2=pd.merge(df,neighborhoods,on= 'Neighborhood')
df2.head()

In [ ]:
source = requests.get('https://github.com/nychealth/coronavirus-data/blob/master/boro.csv').text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

In [ ]:
df4 = pd.read_html(tab)
df=df4[0]
df.head()

In [ ]:
df3=pd.merge(df,neighborhoods,left_on='BOROUGH_GROUP', right_on='Borough')
df3.head()

In [ ]:
df4=df3.dropna(axis='columns')
df4.head()

In [ ]:
df5=df4.drop(columns='BOROUGH_GROUP')
df5.head()

In [ ]:
CLIENT_ID = '402RXOBDXFHKXVGEZSM3USD4FD2YLOZMCHF4JLLUN1KI5G5D' # your Foursquare ID
CLIENT_SECRET = 'YH2BIDZCC21G0Y1YGATQGY553RJOEKZQM5R1DS544YWCD5FZ' # your Foursquare Secret
VERSION = '20200401'

In [ ]:
map_newyork = folium.Map(location=[40.7308619,-73.9871558],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df5['Latitude'],df5['Longitude'],df5['Borough'],df5['COVID_CASE_COUNT']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_newyork)
map_newyork

### **Using Histogram to show the distribution**

In [ ]:
df.head()

In [ ]:
df6=df.drop(columns='COVID_CASE_RATE')
df6.head()

In [ ]:
df7=df6.dropna(axis='columns')
df7.head()

In [ ]:
df7.plot(kind='hist', figsize=(10, 6))

plt.title('Histogram of Immigration from Denmark, Norway, and Sweden from 1980 - 2013')
plt.ylabel('COVID_CASE_COUNT')
plt.xlabel('BOROUGH_GROUP')

plt.show()

In [ ]:
plt.figure(figsize=(15,10))
df7.sort_values(by="points",ascending=False)["points"].plot.bar()
plt.xticks(rotation=50)
plt.xlabel("BOROUGH_GROUP")
plt.ylabel("COVID_CASE_COUNT")
plt.show()

In [ ]:
# install wordcloud
!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

In [ ]:
# download file and save as alice_novel.txt
!wget --quiet https://github.com/nychealth/coronavirus-data/blob/master/boro.csv

# open the file and read it into a variable alice_novel
Boro = open('boro.csv', 'r').read()
    
print ('File downloaded and saved!')

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
# instantiate a word cloud object
boro_wc = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
boro_wc.generate(Boro)

In [ ]:
# display the word cloud
plt.imshow(boro_wc, interpolation='bilinear')
plt.axis('off')

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(45)

plt.show()